In [20]:
import torch
use_cuda = torch.cuda.is_available()
if use_cuda:
    print('__Memory Allocated (Device):',torch.cuda.memory_allocated())
    print('__Max Memory Allocated:',torch.cuda.max_memory_allocated()/1e6)
    print('__Memory Reserved:',torch.cuda.memory_reserved()/1e9)
    print('__Currently used device:', torch.cuda.current_device())
    print('__CUDNN VERSION:', torch.backends.cudnn.version())
    print('__Number CUDA Devices:', torch.cuda.device_count())
    print('__CUDA Device Name:',torch.cuda.get_device_name(0))
    print('__CUDA Device Total Memory [GB]:',torch.cuda.get_device_properties(0))#.total_memory/1e6)

__Memory Allocated (Device): 0
__Max Memory Allocated: 0.0
__Memory Reserved: 0.0
__Currently used device: 0
__CUDNN VERSION: 8500
__Number CUDA Devices: 1
__CUDA Device Name: NVIDIA GeForce RTX 2080 SUPER
__CUDA Device Total Memory [GB]: _CudaDeviceProperties(name='NVIDIA GeForce RTX 2080 SUPER', major=7, minor=5, total_memory=7969MB, multi_processor_count=48)


In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
# !python --version


In [4]:
# !pip3 install text-preprocessing

# !pip3 install nltk
# !pip3 install lexical-diversity
#!pip3 install trunajod
#!pip3 install lexicalrichness

In [22]:
import os
import csv 
import numpy as np
import pandas as pd
import random
import math
import ast
import re
import regex
import string
#import functools
from functools import reduce
import operator  
import warnings

from itertools import groupby
from collections import OrderedDict
from collections import defaultdict
from datetime import datetime
from datetime import date
from ast import literal_eval

import urllib
import requests
import json
import shutil
import time
import fnmatch
from pathlib import Path


#import PyPaperBot
import nltk
import fitz
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize.treebank import TreebankWordDetokenizer

import pickle
from text_preprocessing import preprocess_text
from text_preprocessing import *

from sklearn.metrics.pairwise import cosine_similarity

import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.pyplot import figure


warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)


In [23]:
import seaborn as sns
from scipy import stats
import statsmodels.api as sm
from patsy import dmatrices
import statsmodels.formula.api as smf

from scipy.stats import shapiro
from scipy.stats import normaltest

import pylab 

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectFromModel
from sklearn import model_selection
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn import preprocessing

#!pip3 install sklearn_pandas
from sklearn_pandas import DataFrameMapper
from sklearn.model_selection import cross_val_score


from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from math import sqrt

In [24]:
import textacy
from textacy import text_stats as ts
from textacy import extract
from textacy import preprocessing

import textstat  # https://pypi.org/project/textstat/

#import textract  # To read .docx files
import spacy
import tarfile
import TRUNAJOD.givenness
import TRUNAJOD.ttr
# from TRUNAJOD import surface_proxies
from TRUNAJOD.syllabizer import Syllabizer

from TRUNAJOD import surface_proxies
from TRUNAJOD.entity_grid import EntityGrid
from TRUNAJOD.lexico_semantic_norms import LexicoSemanticNorm

from lexicalrichness import LexicalRichness

from lexical_diversity import lex_div as ld # https://pypi.org/project/lexical-diversity/
#from textacy.text_stats import basics, counts, diversity, readability

In [25]:
nltk.download('stopwords')
nltk.download('words')
# !python -m spacy download en_core_web_sm

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/psych256lab/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     /home/psych256lab/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [27]:
from nltk.corpus import stopwords as sw
#stop_words = stopwords.words('english')
stop_words = sw.words('english')

vocab = set(w.lower() for w in nltk.corpus.words.words())
lemmatizer = WordNetLemmatizer()

In [5]:
class ModelLoader(object):
    """Class to load model."""
    def __init__(self, model_file):
        tar = tarfile.open(model_file, "r:gz")
        self.crea_frequency = {}
        self.infinitive_map = {}
        self.lemmatizer = {}
        self.spanish_lexicosemantic_norms = {}
        self.stopwords = {}
        self.wordnet_noun_synsets = {}
        self.wordnet_verb_synsets = {}

        for member in tar.getmembers():
            f = tar.extractfile(member)
            if "crea_frequency" in member.name:
                self.crea_frequency = pickle.loads(f.read())
            if "infinitive_map" in member.name:
                self.infinitive_map = pickle.loads(f.read())
            if "lemmatizer" in member.name:
                self.lemmatizer = pickle.loads(f.read())
            if "spanish_lexicosemantic_norms" in member.name:
                self.spanish_lexicosemantic_norms = pickle.loads(f.read())
            if "stopwords" in member.name:
                self.stopwords = pickle.loads(f.read())
            if "wordnet_noun_synsets" in member.name:
                self.wordnet_noun_synsets = pickle.loads(f.read())
            if "wordnet_verb_synsets" in member.name:
                self.wordnet_verb_synsets = pickle.loads(f.read())


# Load TRUNAJOD models
model = ModelLoader("trunajod_models_v0.1.tar.gz")

In [11]:
# for reproducibility
np.random.seed(1)

In [28]:
filterd_articlePolicyList2 = pd.read_csv('filterd_articlePolicyList.csv', encoding='utf-8')

In [29]:
filterd_articlePolicyList2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2301 entries, 0 to 2300
Data columns (total 10 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   url                         2301 non-null   object 
 1   title                       2301 non-null   object 
 2   policyList                  2301 non-null   object 
 3   policyTitles                2301 non-null   object 
 4   USApolicyCitation           2301 non-null   int64  
 5   successful_policyList       2301 non-null   object 
 6   successful_policyTitles     2301 non-null   object 
 7   articlePdf                  2301 non-null   object 
 8   total_successful_policy     2301 non-null   int64  
 9   successfulPolicyPercentage  2301 non-null   float64
dtypes: float64(1), int64(2), object(7)
memory usage: 179.9+ KB


In [30]:
filterd_articlePolicyList2.set_index('url', inplace =True)
# filterd_articlePolicyList2.at['https://doi.org/10.1001/archderm.139.3.318','articlePdf']

In [31]:
filterd_articlePolicyList2.policyTitles = filterd_articlePolicyList2.policyTitles.apply(literal_eval)
filterd_articlePolicyList2.policyList =  filterd_articlePolicyList2.policyList.apply(literal_eval)                
filterd_articlePolicyList2.successful_policyList = filterd_articlePolicyList2.successful_policyList.apply(literal_eval)
filterd_articlePolicyList2.successful_policyTitles = filterd_articlePolicyList2.successful_policyTitles.apply(literal_eval)
#filterd_articlePolicyList2.articlePdf = filterd_articlePolicyList2.articlePdf.apply(literal_eval)

In [11]:
new_columns =['char_count','letter_count','word_count', 'syllable_count','monosyllabcount','polysyllabcount','sentence_count',\
              'avg_word_length','average_sentence_length','avg_syllables_per_word',\
              'n_unique_words', 'n_long_words','n_difficult_words','avg_n_unique_words','avg_n_long_words','avg_n_difficult_words', \
              
              'avg_noun_count','avg_verb_count','avg_adverb_count','avg_adjective_count','avg_conjunction_count','avg_adposition_count','imp_pos_ratio',\
              
              'frequency_index','pos_dissimilarity','syntactic_similarity','words_before_root',\
            
              
              'lex_ttr','lex_rttr','lex_cttr','lex_maas','lex_msttr','lex_mattr','lex_hdd','lex_mtld',\
              'lex_vocd','lex_herdan','lex_summer','lex_dugast','lex_yulek','lex_yulei','lex_herdanvm','lex_simpsond',
              
              'reading_time','smog_index','flesch_reading_ease','flesch_kincaid_grade','automated_readability_index','gunning_fog','coleman_liau_index','dale_chall_readability_score','linsear_write_formula','mcalpine_eflaw'
              ]

In [15]:
len(new_columns)

53

In [16]:
for column in new_columns:
    #new_similarity_column = f"{model}_sim"
    filterd_articlePolicyList2[column]=0.0

In [17]:
#filterd_articlePolicyList2.info()

In [18]:
# #Load sentences & embeddings from disc
with open('article_sentences.pkl', "rb") as fIn:
    article_sentences = pickle.load(fIn)
#filterd_articlePolicyList2.info()


In [19]:
# doi = 'https://doi.org/10.9778/cmajo.20190026'

#article_sentences[doi]  

In [20]:
def lexical_analysis(index: str, text:str):
    doc = textacy.make_spacy_doc(text, lang="en_core_web_sm")
    ts1 = ts.TextStats(doc)
    nlp = spacy.load("en_core_web_sm", disable=["ner", "textcat"])
    doc1 = nlp(text)
    
    word_count = textstat.lexicon_count(text, removepunct=True)
    if (word_count>499):
    
        filterd_articlePolicyList2.at[index,'char_count'] = textstat.char_count(text, ignore_spaces=True)
        filterd_articlePolicyList2.at[index,'letter_count'] = textstat.letter_count(text, ignore_spaces=True)
        filterd_articlePolicyList2.at[index,'word_count'] = word_count
        filterd_articlePolicyList2.at[index,'syllable_count'] = textstat.syllable_count(text)
        filterd_articlePolicyList2.at[index,'monosyllabcount'] = textstat.monosyllabcount(text)
        filterd_articlePolicyList2.at[index,'polysyllabcount'] = textstat.polysyllabcount(text)

        filterd_articlePolicyList2.at[index,'sentence_count'] = textstat.sentence_count(text)
        filterd_articlePolicyList2.at[index,'avg_word_length'] = textstat.avg_letter_per_word(text)
        filterd_articlePolicyList2.at[index,'average_sentence_length'] = textstat.avg_sentence_length(text)
        filterd_articlePolicyList2.at[index,'avg_syllables_per_word'] =  textstat.avg_syllables_per_word(text)

        n_unique_words = ts1.n_unique_words
        n_long_words = ts1.n_long_words
        n_difficult_words = textstat.difficult_words(text)

        filterd_articlePolicyList2.at[index,'n_unique_words'] = float("{:0.3f}".format(n_unique_words))
        filterd_articlePolicyList2.at[index,'n_long_words'] =  float("{:0.3f}".format(n_long_words))
        filterd_articlePolicyList2.at[index,'n_difficult_words'] = float("{:0.3f}".format(n_difficult_words))

        filterd_articlePolicyList2.at[index,'avg_n_unique_words'] =  float("{:0.3f}".format(n_unique_words/word_count))
        filterd_articlePolicyList2.at[index,'avg_n_long_words'] =  float("{:0.3f}".format(n_long_words/word_count))
        filterd_articlePolicyList2.at[index,'avg_n_difficult_words'] =  float("{:0.3f}".format(n_difficult_words/word_count))

        pos_dist = surface_proxies.pos_distribution(doc1)

        noun_count = pos_dist['NOUN'] #surface_proxies.noun_count(doc))
        adjective_count = pos_dist['ADJ']
        verb_count = pos_dist['VERB']
        adverb_count = pos_dist['ADV']
        adposition_count = pos_dist['ADP']
        conjunction_count = pos_dist['CCONJ']+pos_dist['SCONJ']

        filterd_articlePolicyList2.at[index,'avg_noun_count'] = float("{:0.3f}".format(noun_count/word_count))
        filterd_articlePolicyList2.at[index,'avg_adjective_count'] = float("{:0.3f}".format(adjective_count/word_count))
        filterd_articlePolicyList2.at[index,'avg_verb_count'] = float("{:0.3f}".format(verb_count/word_count))
        filterd_articlePolicyList2.at[index,'avg_adverb_count'] =  float("{:0.3f}".format(adverb_count/word_count))
        filterd_articlePolicyList2.at[index,'avg_adposition_count'] =  float("{:0.3f}".format(adposition_count/word_count))
        filterd_articlePolicyList2.at[index,'avg_conjunction_count'] =  float("{:0.3f}".format(conjunction_count/word_count))

        filterd_articlePolicyList2.at[index,'imp_pos_ratio'] =  float("{:0.3f}".format((noun_count+adjective_count+verb_count+adverb_count) / word_count))

        filterd_articlePolicyList2.at[index,'frequency_index'] =  float("{:0.3f}".format(surface_proxies.frequency_index(doc1, model.crea_frequency)))
        filterd_articlePolicyList2.at[index,'syntactic_similarity'] = float("{:0.3f}".format( surface_proxies.syntactic_similarity(doc1)))
        filterd_articlePolicyList2.at[index,'words_before_root'] = float("{:0.3f}".format( surface_proxies.words_before_root(doc1,max_depth=4)))
        filterd_articlePolicyList2.at[index,'pos_dissimilarity'] = float("{:0.3f}".format( surface_proxies.pos_dissimilarity(doc1)))
    
#     filterd_articlePolicyList2.at[index,'word_variation_index'] =  TRUNAJOD.ttr.word_variation_index(doc1) 
   
#     filterd_articlePolicyList2.at[index,'ttr_d_estimate'] =  TRUNAJOD.ttr.d_estimate(doc1, min_range = 35, max_range = 50, trials = 5) 
#     filterd_articlePolicyList2.at[index,'ttr_yule_k'] =  TRUNAJOD.ttr.yule_k(doc1)

        lex = LexicalRichness(text)

        # # Return word count.
        # filterd_articlePolicyList2.at[index,'words'] =  lex.words

        # # Return (unique) word count.
        # filterd_articlePolicyList2.at[index,'unique_words'] =  lex.terms

        # Return type-token ratio (TTR) of text.
        filterd_articlePolicyList2.at[index,'lex_ttr'] = float("{:0.3f}".format( lex.ttr))
        # Return root type-token ratio (RTTR) of text.
        filterd_articlePolicyList2.at[index,'lex_rttr'] =  float("{:0.3f}".format(lex.rttr))
        # Return corrected type-token ratio (CTTR) of text.
        filterd_articlePolicyList2.at[index,'lex_cttr'] = float("{:0.3f}".format( lex.cttr))
        # Return Maas's lexical diversity measure.
        filterd_articlePolicyList2.at[index,'lex_maas'] = float("{:0.3f}".format( lex.Maas))
        # Return hypergeometric distribution diversity (HD-D) measure.
        filterd_articlePolicyList2.at[index,'lex_hdd'] =  float("{:0.3f}".format(lex.hdd(draws=42)))
        # Return mean segmental type-token ratio (MSTTR).
        filterd_articlePolicyList2.at[index,'lex_msttr'] = float("{:0.3f}".format( lex.msttr(segment_window=25)))
        # Return moving average type-token ratio (MATTR).
        filterd_articlePolicyList2.at[index,'lex_mattr'] =  float("{:0.3f}".format(lex.mattr(window_size=25)))
        # Return Measure of Textual Lexical Diversity (MTLD).
        filterd_articlePolicyList2.at[index,'lex_mtld'] =  float("{:0.3f}".format(lex.mtld(threshold=0.72)))
        # Return voc-D measure.
        filterd_articlePolicyList2.at[index,'lex_vocd'] = float("{:0.3f}".format( lex.vocd(ntokens=50, within_sample=100, iterations=3)))
        # Return Herdan's lexical diversity measure.
        filterd_articlePolicyList2.at[index,'lex_herdan'] =  float("{:0.3f}".format(lex.Herdan))
        # Return Summer's lexical diversity measure.
        filterd_articlePolicyList2.at[index,'lex_summer'] =  float("{:0.3f}".format(lex.Summer))
        # Return Dugast's lexical diversity measure.
        filterd_articlePolicyList2.at[index,'lex_dugast'] =  float("{:0.3f}".format(lex.Dugast))
        # Return Yule's K.
        filterd_articlePolicyList2.at[index,'lex_yulek'] =  float("{:0.3f}".format(lex.yulek))
        # Return Yule's I.
        filterd_articlePolicyList2.at[index,'lex_yulei'] = float("{:0.3f}".format( lex.yulei))
        # Return Herdan's Vm.
        filterd_articlePolicyList2.at[index,'lex_herdanvm'] =  float("{:0.3f}".format(lex.herdanvm))
        # Return Simpson's D.
        filterd_articlePolicyList2.at[index,'lex_simpsond'] = float("{:0.3f}".format( lex.simpsond))

        # Readability

        filterd_articlePolicyList2.at[index,'reading_time'] =  float("{:0.3f}".format(textstat.reading_time(text, ms_per_char=14.69)))
        filterd_articlePolicyList2.at[index,'flesch_reading_ease'] =  float("{:0.3f}".format(textstat.flesch_reading_ease(text)))
        filterd_articlePolicyList2.at[index,'flesch_kincaid_grade'] =  float("{:0.3f}".format(textstat.flesch_kincaid_grade(text)))
        filterd_articlePolicyList2.at[index,'smog_index'] =  float("{:0.3f}".format(textstat.smog_index(text)))
        filterd_articlePolicyList2.at[index,'coleman_liau_index'] =  float("{:0.3f}".format(textstat.coleman_liau_index(text)))
        filterd_articlePolicyList2.at[index,'automated_readability_index'] =  float("{:0.3f}".format(textstat.automated_readability_index(text)))
        filterd_articlePolicyList2.at[index,'dale_chall_readability_score'] =  float("{:0.3f}".format(textstat.dale_chall_readability_score(text)))

        filterd_articlePolicyList2.at[index,'linsear_write_formula'] = float("{:0.3f}".format( textstat.linsear_write_formula(text)))
        filterd_articlePolicyList2.at[index,'gunning_fog'] =  float("{:0.3f}".format(textstat.gunning_fog(text)))
        filterd_articlePolicyList2.at[index,'mcalpine_eflaw'] =  float("{:0.3f}".format(textstat.mcalpine_eflaw(text)))
    
    return None

#     #lexical-diversity
    #flt = ld.flemmatize(text)
    # print("no of tokens: ",len(ld.tokenize(text)))
    # filterd_articlePolicyList2.at[index,'ttr'] =  ld.ttr(flt)
    # filterd_articlePolicyList2.at[index,'root_ttr'] =  ld.root_ttr(flt)
    # filterd_articlePolicyList2.at[index,'log_ttr'] =  ld.log_ttr(flt)
    # filterd_articlePolicyList2.at[index,'maas_ttr'] =  ld.maas_ttr(flt)
    # filterd_articlePolicyList2.at[index,'hdd'] =  ld.hdd(flt)
    # filterd_articlePolicyList2.at[index,'msttr'] =  ld.msttr(flt,window_length=25)
    # filterd_articlePolicyList2.at[index,'mattr'] =  ld.mattr(flt,window_length=25)
    # filterd_articlePolicyList2.at[index,'mtld'] =  ld.mtld(flt)
    # filterd_articlePolicyList2.at[index,'mtld_ma_wrap'] =  ld.mtld_ma_wrap(flt)
    # filterd_articlePolicyList2.at[index,'mtld_ma_bid'] =  ld.mtld_ma_bid(flt)


In [21]:
remove_token = list(string.ascii_lowercase[1:26]) +['al','mo','cox']

In [22]:
def sentenceProcessing(sentence:str)->list:
    token_list = nltk.word_tokenize(sentence.lower())
    filtered_token_list = [token for token in token_list if token not in remove_token if token in vocab ] #
    filtered_token_list1 = [i[0] for i in groupby(filtered_token_list)] #removes consecutive duplicate words
    filtered_token_list2 = [token for token in filtered_token_list1 if token not in stop_words]
    if len(filtered_token_list2)!=0 and len(filtered_token_list1)>2:
        return filtered_token_list1
    else:
        return list()

In [23]:
def textProcessing(index: str):  # article is a list of sentences
    if index in article_sentences:
        article_sentence_list = article_sentences[index]
        if (len(article_sentence_list)!=0):
            tokenized_sentence_list = [sentenceProcessing(sentence) for sentence in article_sentence_list if sentence != ''] #doc.split()
            preprocessed_article= ' '.join([TreebankWordDetokenizer().detokenize(tokenized_sentence)+'.' for tokenized_sentence in tokenized_sentence_list if len(tokenized_sentence)!=0])

            lexical_analysis(index, preprocessed_article)
    ### call function for measurements
    
    return None


In [24]:
article_list = list(filterd_articlePolicyList2.index)
# textProcessing(doi, article_sentences[doi])

In [25]:
docs=[textProcessing(article) for article in article_list] # if article_list.index(article)>1337]

In [27]:
article_list.index('https://doi.org/10.1016/s1470-2045(11)70336-9')

1339

In [26]:
len(article_sentences)

2296

In [28]:
article_list[329]

'https://doi.org/10.1001/jamapediatrics.2015.3778'

In [ ]:
article_sentences[article_list[1339]]

In [ ]:
filterd_articlePolicyList2.tail()

In [ ]:
# filterd_articlePolicyList2.loc[article_list[500]:,new_columns[0:20]] 

In [ ]:
# filterd_articlePolicyList2.loc[article_list[1337]:,new_columns[20:40]]

In [337]:
# filterd_articlePolicyList2.loc[doi:,new_columns[40:]]

In [347]:
filterd_articlePolicyList2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2301 entries, https://doi.org/10.1001/archderm.138.12.1584 to https://doi.org/10.9778/cmajo.20190026
Data columns (total 62 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   title                         2301 non-null   object 
 1   policyList                    2301 non-null   object 
 2   policyTitles                  2301 non-null   object 
 3   USApolicyCitation             2301 non-null   int64  
 4   successful_policyList         2301 non-null   object 
 5   successful_policyTitles       2301 non-null   object 
 6   articlePdf                    2301 non-null   object 
 7   total_successful_policy       2301 non-null   int64  
 8   successfulPolicyPercentage    2301 non-null   float64
 9   char_count                    2301 non-null   float64
 10  letter_count                  2301 non-null   float64
 11  word_count                    2301 non-null   flo

In [348]:
len(filterd_articlePolicyList2)

2301

In [349]:
final_csv_file = filterd_articlePolicyList2[filterd_articlePolicyList2['word_count']!=0]

In [350]:
len(final_csv_file)

2271

In [355]:
#posion dist
final_csv_file1 = final_csv_file.copy(deep = False)
# final_csv_file1.reset_index(inplace = True)

In [ ]:
final_csv_file1.info()

In [ ]:
# final_csv_file1.head()

### updated tfidf file

In [357]:
def avgSimCalculation(cellValue:list)->float:
    newValue = float("{:0.3f}".format(np.mean(cellValue)))
    return newValue


In [358]:
updated_tfidfScores_file = pd.read_csv('updated_tfidfScores.csv', usecols =['url','tfidf_sim'], encoding = 'utf-8')
updated_tfidfScores_file.tfidf_sim = updated_tfidfScores_file.tfidf_sim.apply(literal_eval)

updated_tfidfScores_file['mean_tfidf_sim'] = updated_tfidfScores_file['tfidf_sim'].map(avgSimCalculation)
updated_tfidfScores_file.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2301 entries, 0 to 2300
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   url             2301 non-null   object 
 1   tfidf_sim       2301 non-null   object 
 2   mean_tfidf_sim  2280 non-null   float64
dtypes: float64(1), object(2)
memory usage: 54.1+ KB


### File with mean article-policy similarity scores

In [14]:
mean_sim_file_columns = ['url','mean_bert_base_uncased_sim', 'mean_sentence_t5_base_sim','mean_biobert_large_cased_v11_squad_sim',
       'mean_biobert_large_cased_sim', 'mean_biobert_base_cased_v11_sim', 'mean_biobert_v11_sim','mean_bert_large_uncased_sim',
       'mean_all_distilroberta_v1_sim', 'mean_all_mpnet_base_v2_sim',
       'mean_all_MiniLM_L6_v2_sim', 
       'mean_sentence_t5_xl_sim', 'mean_political_bert_sim',
       'mean_specter_sim', 'mean_clinical_bert_sim', 'mean_legal_bert_sim',
       'mean_scibert_sim', 
       'mean_biobert_base_cased_v12_sim']

In [17]:
file_with_meanSimScores = pd.read_csv('finalFile_with_meanSimScores.csv', usecols = mean_sim_file_columns, encoding = 'utf-8')

In [19]:
file_with_meanSimScores

,url,mean_bert_base_uncased_sim,mean_bert_large_uncased_sim,mean_all_distilroberta_v1_sim,mean_all_mpnet_base_v2_sim,mean_all_MiniLM_L6_v2_sim,mean_sentence_t5_base_sim,mean_sentence_t5_xl_sim,mean_political_bert_sim,mean_specter_sim,mean_clinical_bert_sim,mean_legal_bert_sim,mean_scibert_sim,mean_biobert_large_cased_v11_squad_sim,mean_biobert_large_cased_sim,mean_biobert_v11_sim,mean_biobert_base_cased_v12_sim,mean_biobert_base_cased_v11_sim
0,https://doi.org/10.1001/archderm.138.12.1584,0.480,0.600,0.151,0.162,0.135,0.743,0.645,0.579,0.649,0.782,0.659,0.580,0.913,0.791,0.723,0.772,0.763
1,https://doi.org/10.1001/archderm.139.3.318,0.504,0.585,0.143,0.153,0.128,0.731,0.635,0.565,0.670,0.769,0.683,0.587,0.901,0.788,0.725,0.762,0.745
2,https://doi.org/10.1001/archgenpsychiatry.2009...,0.498,0.593,0.163,0.185,0.157,0.735,0.642,0.557,0.687,0.776,0.685,0.599,0.913,0.789,0.733,0.767,0.755
3,https://doi.org/10.1001/archgenpsychiatry.2009...,0.475,0.572,0.113,0.128,0.107,0.724,0.623,0.537,0.676,0.753,0.681,0.602,0.903,0.792,0.716,0.752,0.742
4,https://doi.org/10.1001/archgenpsychiatry.2010...,0.526,0.608,0.183,0.199,0.161,0.739,0.650,0.594,0.708,0.794,0.718,0.583,0.919,0.817,0.744,0.788,0.776
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2296,https://doi.org/10.9745/ghsp-d-18-00275,0.531,0.620,0.157,0.167,0.143,0.734,0.642,0.599,0.689,0.797,0.709,0.612,0.921,0.839,0.751,0.797,0.776
2297,https://doi.org/10.9745/ghsp-d-19-00195,0.560,0.640,0.168,0.175,0.146,0.731,0.641,0.640,0.665,0.816,0.748,0.638,0.928,0.854,0.784,0.822,0.801
2298,https://doi.org/10.9758/cpn.2013.11.3.107,0.507,0.604,0.143,0.145,0.106,0.736,0.645,0.593,0.670,0.787,0.709,0.590,0.917,0.817,0.738,0.781,0.761
2299,https://doi.org/10.9758/cpn.2013.11.3.118,0.583,0.668,0.244,0.224,0.185,0.756,0.682,0.651,0.685,0.828,0.764,0.650,0.924,0.846,0.795,0.824,0.807


### merging Files

In [ ]:
final_csv_file1_columns = list(final_csv_file1.columns)
final_csv_file1_columns

In [375]:
updated_tfidfScores_file1 = updated_tfidfScores_file.set_index('url')

In [376]:
file_with_meanSimScores1 = file_with_meanSimScores.set_index('url')

In [377]:
# merge on common index
merged_file = pd.merge(pd.merge(final_csv_file1[[final_csv_file1_columns[3]]+final_csv_file1_columns[8:]],updated_tfidfScores_file1['mean_tfidf_sim'], left_index =True, right_index = True), file_with_meanSimScores1, left_index =True, right_index = True)
print(len(merged_file))

2271


In [378]:
# https://datatofish.com/check-nan-pandas-dataframe/ 

merged_file1 = merged_file[merged_file['successfulPolicyPercentage']>=70]
print(merged_file1.isnull().values.any()) #merged_file1['mean_tfidf_sim'].isnull().sum()
merged_file2 = merged_file1.copy(deep = False)
merged_file2.dropna(inplace = True)
print(merged_file2.isnull().values.any())

True
False


In [395]:
# merged_file2.reset_index(inplace = True)
# merged_file2.to_csv('merged_file2.csv', encoding = 'utf-8', index = False)

In [12]:
merged_file2 = pd.read_csv('merged_file2.csv', encoding = 'utf-8')

In [13]:
merged_file2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1633 entries, 0 to 1632
Data columns (total 74 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   url                                     1633 non-null   object 
 1   USApolicyCitation                       1633 non-null   int64  
 2   successfulPolicyPercentage              1633 non-null   float64
 3   char_count                              1633 non-null   float64
 4   letter_count                            1633 non-null   float64
 5   word_count                              1633 non-null   float64
 6   syllable_count                          1633 non-null   float64
 7   monosyllabcount                         1633 non-null   float64
 8   polysyllabcount                         1633 non-null   float64
 9   sentence_count                          1633 non-null   float64
 10  avg_word_length                         1633 non-null   floa

In [15]:
mean_sim_file_columns

['url',
 'mean_bert_base_uncased_sim',
 'mean_sentence_t5_base_sim',
 'mean_biobert_large_cased_v11_squad_sim',
 'mean_biobert_large_cased_sim',
 'mean_biobert_base_cased_v11_sim',
 'mean_biobert_v11_sim',
 'mean_bert_large_uncased_sim',
 'mean_all_distilroberta_v1_sim',
 'mean_all_mpnet_base_v2_sim',
 'mean_all_MiniLM_L6_v2_sim',
 'mean_sentence_t5_xl_sim',
 'mean_political_bert_sim',
 'mean_specter_sim',
 'mean_clinical_bert_sim',
 'mean_legal_bert_sim',
 'mean_scibert_sim',
 'mean_biobert_base_cased_v12_sim']

In [22]:
(merged_file2[['mean_tfidf_sim']+mean_sim_file_columns[7:]].mean()>=0.70) # excluding 'url'

mean_tfidf_sim                     False
mean_bert_large_uncased_sim        False
mean_all_distilroberta_v1_sim      False
mean_all_mpnet_base_v2_sim         False
mean_all_MiniLM_L6_v2_sim          False
mean_sentence_t5_xl_sim            False
mean_political_bert_sim            False
mean_specter_sim                   False
mean_clinical_bert_sim              True
mean_legal_bert_sim                 True
mean_scibert_sim                   False
mean_biobert_base_cased_v12_sim     True
dtype: bool

In [51]:
excluding_features = ['mean_tfidf_sim', 'mean_bert_large_uncased_sim', 'mean_all_distilroberta_v1_sim', 'mean_all_mpnet_base_v2_sim', 'mean_all_MiniLM_L6_v2_sim', 'mean_sentence_t5_xl_sim', 'mean_political_bert_sim','mean_specter_sim','mean_scibert_sim']+ mean_sim_file_columns[1:7]
# including_features = ['mean_bert_base_uncased_sim','mean_sentence_t5_base_sim','mean_clinical_bert_sim','mean_legal_bert_sim','mean_biobert_large_cased_v11_squad_sim','mean_biobert_large_cased_sim','mean_biobert_v11_sim','mean_biobert_base_cased_v12_sim','mean_biobert_base_cased_v11_sim']
including_features = ['mean_clinical_bert_sim','mean_legal_bert_sim','mean_biobert_base_cased_v12_sim']
print(len(excluding_features), len(including_features))

15 3


In [52]:
new_columns3 = list(set(merged_file2.columns) - set(excluding_features)-{'successfulPolicyPercentage'})
merged_file3 = merged_file2[new_columns3]

In [53]:
# merged_file2.info()
# m = merged_file2.reset_index()
# m.info()
len(merged_file3)

1633

In [54]:
merged_file3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1633 entries, 0 to 1632
Data columns (total 58 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   lex_herdan                       1633 non-null   float64
 1   avg_noun_count                   1633 non-null   float64
 2   polysyllabcount                  1633 non-null   float64
 3   avg_n_unique_words               1633 non-null   float64
 4   frequency_index                  1633 non-null   float64
 5   avg_adposition_count             1633 non-null   float64
 6   mean_clinical_bert_sim           1633 non-null   float64
 7   lex_simpsond                     1633 non-null   float64
 8   n_long_words                     1633 non-null   float64
 9   lex_summer                       1633 non-null   float64
 10  pos_dissimilarity                1633 non-null   float64
 11  lex_rttr                         1633 non-null   float64
 12  lex_dugast          

#### merged_file2 : all features
#### merged_file3 : similarity scores>=70%

In [653]:
# m.to_csv("merged_file2.csv", encoding = 'utf-8', index = False)

In [32]:
# x = set(sorted(merged_file3.columns))
# y = set(sorted(X_train.columns))
# print(len(x)-len(y))
# print(y-x)

In [57]:
merged_file4= merged_file3.set_index('url')

In [71]:
merged_file4.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1633 entries, https://doi.org/10.1001/archderm.138.12.1584 to https://doi.org/10.9778/cmajo.20190026
Data columns (total 57 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   lex_herdan                       1633 non-null   float64
 1   avg_noun_count                   1633 non-null   float64
 2   polysyllabcount                  1633 non-null   float64
 3   avg_n_unique_words               1633 non-null   float64
 4   frequency_index                  1633 non-null   float64
 5   avg_adposition_count             1633 non-null   float64
 6   mean_clinical_bert_sim           1633 non-null   float64
 7   lex_simpsond                     1633 non-null   float64
 8   n_long_words                     1633 non-null   float64
 9   lex_summer                       1633 non-null   float64
 10  pos_dissimilarity                1633 non-null   float64
 11  lex_rttr  

In [12]:
USAcitations_data= pd.read_csv("USAcitations.csv", encoding="utf-8")

In [30]:
USAcitations_data.columns

Index(['Title', 'DOI', 'Journal', 'Published on', 'Policy citation count',
       'URL', 'Cited by source ID', 'Cited by source', 'Cited by title',
       'Cited by date', 'Cited by type', 'Cited by subtype',
       'Cited by country', 'Cited by URL', 'Document page',
       'Article funder(s)', 'USAcitationCount', 'citationLag',
       'meanCitationLag', 'minCitationLag', 'maxCitationLag'],
      dtype='object')

In [56]:
USAcitations_data1 = USAcitations_data[['URL','maxCitationLag']].drop_duplicates()
USAcitations_data1.set_index('URL', inplace =True)
USAcitations_data1.info()
# ['USApolicyCitation','url'] ['URL','USAcitationCount']

<class 'pandas.core.frame.DataFrame'>
Index: 56415 entries, https://doi.org/10.1080/10459880903217895 to https://doi.org/10.1111/j.1469-7610.2010.02222.x
Data columns (total 1 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   maxCitationLag  56415 non-null  int64
dtypes: int64(1)
memory usage: 881.5+ KB


In [60]:
# merge on common index
merged_file_wt_citationLag = pd.merge(merged_file4, USAcitations_data1, left_index =True, right_index = True)
print(len(merged_file_wt_citationLag))

1633


In [65]:
merged_file_wt_citationLag.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1633 entries, https://doi.org/10.1001/archderm.138.12.1584 to https://doi.org/10.9778/cmajo.20190026
Data columns (total 58 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   lex_herdan                       1633 non-null   float64
 1   avg_noun_count                   1633 non-null   float64
 2   polysyllabcount                  1633 non-null   float64
 3   avg_n_unique_words               1633 non-null   float64
 4   frequency_index                  1633 non-null   float64
 5   avg_adposition_count             1633 non-null   float64
 6   mean_clinical_bert_sim           1633 non-null   float64
 7   lex_simpsond                     1633 non-null   float64
 8   n_long_words                     1633 non-null   float64
 9   lex_summer                       1633 non-null   float64
 10  pos_dissimilarity                1633 non-null   float64
 11  lex_rttr  

In [ ]:
print("mean: ", merged_file3.USApolicyCitation.mean())
print("variance: ",merged_file3.USApolicyCitation.var())
# mean< variance: overdispersion

mean:  2.0777709736680956
variance:  3.7494626754559133


### Feature Scaling

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

# from sklearn.decomposition import PCA

# from scipy.stats import spearmanr

In [ ]:
features_with_minmax_scaling = ['avg_syllables_per_word','n_long_words','lex_mtld','lex_rttr','lex_cttr','automated_readability_index','lex_yulek','flesch_reading_ease', 'mcalpine_eflaw',
                                'flesch_kincaid_grade','polysyllabcount','word_count','syllable_count','sentence_count','char_count','words_before_root','avg_word_length','average_sentence_length',
                                'lex_yulei','reading_time','n_difficult_words','lex_vocd','n_unique_words','lex_dugast','smog_index','gunning_fog','linsear_write_formula',
                                'letter_count','monosyllabcount','coleman_liau_index','dale_chall_readability_score']
len(features_with_minmax_scaling)

In [ ]:
scaler = MinMaxScaler()

In [836]:
scaled_merged_file3 = merged_file3.copy(deep = False)

In [837]:
scaled_merged_file3[features_with_minmax_scaling] = scaler.fit_transform(scaled_merged_file3[features_with_minmax_scaling])

In [ ]:
# scaled_merged_file3.info()
scaled_merged_file3['avg_word_length'].describe()

In [ ]:
# len(new_columns3)
# scaled_merged_file3.info()
len(scaled_merged_file3.columns)

## Classification

In [839]:
scaled_merged_file3_class = scaled_merged_file3.copy(deep = False)

In [840]:
scaled_merged_file3_class.loc[scaled_merged_file3_class['USApolicyCitation']>1, 'USApolicyCitation'] = 0
# df.loc[df["gender"] == "male", "gender"] = 1

In [842]:
scaled_merged_file3_class['USApolicyCitation'].value_counts()

1    1002
0     631
Name: USApolicyCitation, dtype: int64

In [859]:
# explore the algorithm wrapped by RFE
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Perceptron
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import Pipeline
from matplotlib import pyplot



# get a list of models to evaluate
def get_models():
    models = dict()
    # # lr
    # rfe = RFE(estimator=LogisticRegression(), n_features_to_select=5)
    # model = DecisionTreeClassifier()
    # models['lr'] = Pipeline(steps=[('s',rfe),('m',model)])
    # perceptron
    rfe = RFE(estimator=Perceptron(), n_features_to_select=10)
    model = DecisionTreeClassifier()
    models['per'] = Pipeline(steps=[('s',rfe),('m',model)])
    # cart
    rfe = RFE(estimator=DecisionTreeClassifier(), n_features_to_select=10)
    model = DecisionTreeClassifier()
    models['cart'] = Pipeline(steps=[('s',rfe),('m',model)])
    # rf
    rfe = RFE(estimator=RandomForestClassifier(), n_features_to_select=10)
    model = DecisionTreeClassifier()
    models['rf'] = Pipeline(steps=[('s',rfe),('m',model)])
    # gbm
    rfe = RFE(estimator=GradientBoostingClassifier(), n_features_to_select=10)
    model = DecisionTreeClassifier()
    models['gbm'] = Pipeline(steps=[('s',rfe),('m',model)])
    return models

# evaluate a give model using cross-validation
def evaluate_model(model, X, y):
    cv = RepeatedStratifiedKFold(n_splits=3, n_repeats=3, random_state=1)
    scores_pre = cross_val_score(model, X, y, scoring='precision', cv=cv, n_jobs=-1)
    scores_re = cross_val_score(model, X, y, scoring='recall', cv=cv, n_jobs=-1)
    return (scores_pre,scores_re)

# define dataset
X = scaled_merged_file3_class.drop(['USApolicyCitation'], axis = 1)
y = scaled_merged_file3_class['USApolicyCitation']
# X_train = sm.add_constant(X_train)
# get the models to evaluate
models = get_models()
# evaluate the models and store results
results, names = list(), list()
for name, model in models.items():
    scores_pre, scores_re = evaluate_model(model, X, y)
    
    names.append(name)
    print('>%s %.3f (%.3f)' % (name, mean(scores_pre), std(scores_pre)))
    print('>%s %.3f (%.3f)' % (name, mean(scores_re), std(scores_re)))
    
    f1_score = (2*mean(scores_pre)*mean(scores_re))/(mean(scores_pre)+mean(scores_re))
    
    print(f'{name}: {f1_score}')
    
    
    # results.append(f1_score)
# plot model performance for comparison
# pyplot.boxplot(results, labels=names, showmeans=True)
# pyplot.show()

>per 0.675 (0.018)
>per 0.658 (0.022)
per: 0.6664872869143118
>cart 0.664 (0.014)
>cart 0.654 (0.027)
cart: 0.6589370824741849
>rf 0.654 (0.016)
>rf 0.644 (0.026)
rf: 0.6491756005960523
>gbm 0.668 (0.018)
>gbm 0.661 (0.031)
gbm: 0.664349253106626


## dataset split

In [ ]:
print(len(merged_file3.columns))
sorted(list(merged_file3.columns))

In [795]:
df_train,df_test=train_test_split(scaled_merged_file3, train_size = 0.7,random_state =1) #new_frame
# X_train = train['mean_tfidf_sim'].values.reshape(-1, 1)
# y_train = train.USApolicyCitation
# X_train.shape,y_train.shape

In [797]:
print(len(df_train.columns))

62


In [798]:

#Setup the regression expression in patsy notation. We are telling patsy that BB_COUNT is our dependent variable and
# it depends on the regression variables: DAY, DAY_OF_WEEK, MONTH, HIGH_T, LOW_T and PRECIP.
# expr = """USApolicyCitation ~ avg_n_long_words+avg_n_difficult_words+avg_adposition_count+pos_dissimilarity+syntactic_similarity+lex_mtld"""
# expr = """USApolicyCitation ~ avg_n_unique_words+char_count+letter_count+word_count+syllable_count+monosyllabcount+polysyllabcount+sentence_count+avg_word_length+average_sentence_length+avg_syllables_per_word+n_unique_words+n_long_words+n_difficult_words+avg_n_unique_words+avg_n_long_words+avg_n_difficult_words+avg_noun_count+avg_verb_count+avg_adverb_count+avg_adjective_count+avg_conjunction_count+avg_adposition_count+imp_pos_ratio+frequency_index+pos_dissimilarity+syntactic_similarity+words_before_root+lex_ttr+lex_rttr+lex_cttr+lex_maas+lex_msttr+lex_mattr+lex_hdd+lex_mtld+lex_vocd+lex_herdan+lex_summer+lex_dugast+lex_yulek+lex_yulei+lex_herdanvm+lex_simpsond+reading_time+smog_index+flesch_reading_ease+flesch_kincaid_grade+automated_readability_index+gunning_fog+coleman_liau_index+dale_chall_readability_score+linsear_write_formula+
#  mcalpine_eflaw+linsear_write_formula"""

# expr = """USApolicyCitation ~ avg_n_unique_words+char_count+letter_count+word_count+syllable_count+monosyllabcount+polysyllabcount+sentence_count+avg_word_length+average_sentence_length+avg_syllables_per_word+n_unique_words+n_long_words+n_difficult_words+avg_n_unique_words+avg_n_long_words+avg_n_difficult_words+avg_noun_count+avg_verb_count+avg_adverb_count+avg_adjective_count+avg_conjunction_count+avg_adposition_count+imp_pos_ratio+frequency_index+pos_dissimilarity+syntactic_similarity+words_before_root+lex_ttr+lex_rttr+lex_cttr+lex_maas+lex_msttr+lex_mattr+lex_hdd+lex_mtld+lex_vocd+lex_herdan+lex_summer+lex_dugast+lex_yulek+lex_yulei+lex_herdanvm+lex_simpsond+reading_time+smog_index+flesch_reading_ease+flesch_kincaid_grade+automated_readability_index+gunning_fog+coleman_liau_index+dale_chall_readability_score+linsear_write_formula+
  # mcalpine_eflaw+linsear_write_formula+mean_bert_base_uncased_sim+mean_bert_large_uncased_sim+mean_all_distilroberta_v1_sim+mean_all_mpnet_base_v2_sim+mean_all_MiniLM_L6_v2_sim+mean_sentence_t5_base_sim+ mean_tfidf_sim+
  # +mean_sentence_t5_xl_sim+mean_political_bert_sim+mean_specter_sim+mean_clinical_bert_sim+mean_legal_bert_sim+mean_scibert_sim+mean_biobert_large_cased_v11_squad_sim+mean_biobert_large_cased_sim+mean_biobert_v11_sim+mean_biobert_base_cased_v12_sim+mean_biobert_base_cased_v11_sim"""
#Set up the X and y matrices
# expr = """USApolicyCitation ~ avg_verb_count+linsear_write_formula+mean_specter_sim+syntactic_similarity+avg_adjective_count+avg_adposition_count+pos_dissimilarity+words_before_root
# +avg_conjunction_count+frequency_index+mean_tfidf_sim+avg_adverb_count"""


expr = """USApolicyCitation ~ automated_readability_index+
                             avg_n_unique_words+
                             avg_syllables_per_word+
                             avg_n_long_words+
                             avg_n_difficult_words+
                             avg_n_unique_words+
                             average_sentence_length+
                             avg_word_length+
                             
                             avg_noun_count+
                             avg_verb_count+
                             avg_adverb_count+
                             avg_adjective_count+
                             avg_conjunction_count+
                             avg_adposition_count+
                             
                             char_count+
                             coleman_liau_index+ 
                             
                             dale_chall_readability_score+
                             
                             flesch_reading_ease+
                             flesch_kincaid_grade+
                             frequency_index+  
                             
                             gunning_fog+
                             
                             imp_pos_ratio+
                             
                             lex_ttr+
                             lex_rttr+
                             lex_cttr+
                             lex_maas+
                             lex_msttr+
                             lex_mattr+
                             lex_hdd+
                             lex_mtld+
                             lex_vocd+
                             lex_herdan+
                             lex_summer+
                             lex_dugast+
                             lex_yulek+
                             lex_yulei+
                             lex_herdanvm+
                             lex_simpsond+
                             letter_count+
                             linsear_write_formula+
                             
                             mcalpine_eflaw+
                             mean_sentence_t5_base_sim+
                             mean_clinical_bert_sim+
                             mean_legal_bert_sim+
                             mean_biobert_large_cased_v11_squad_sim+
                             mean_biobert_large_cased_sim+
                             mean_biobert_v11_sim+
                             mean_biobert_base_cased_v12_sim+
                             mean_biobert_base_cased_v11_sim+
                             monosyllabcount+
                            
                             n_unique_words+
                             n_long_words+
                             n_difficult_words+
                            
                             polysyllabcount+
                             pos_dissimilarity+
                            
                             reading_time+
                             
                             sentence_count+
                             smog_index+
                             syntactic_similarity+
                             syllable_count+
                            
                             word_count+
                             words_before_root
                            
                
                              
                            """
#Set up the X and y matrices

y_train, X_train = dmatrices(expr, df_train, return_type='dataframe')
y_test, X_test = dmatrices(expr, df_test, return_type='dataframe')


In [800]:
# len(df_train.columns)

In [598]:
# X_train = df_train.drop(['USApolicyCitation'], axis = 1)
# X_train = sm.add_constant(X_train)

# y_train = df_train['USApolicyCitation']

# X_test =  df_test.drop(['USApolicyCitation'], axis = 1)
# X_test = sm.add_constant(X_test)

# y_test = df_test['USApolicyCitation']

#### Dimensionality Reduction

In [762]:
# len(X_train.columns)

## Feature selection

In [723]:
merged_file3.columns

Index(['lex_herdanvm', 'avg_verb_count', 'avg_word_length', 'lex_hdd',
       'word_count', 'flesch_kincaid_grade', 'lex_rttr',
       'dale_chall_readability_score', 'lex_cttr', 'mcalpine_eflaw',
       'pos_dissimilarity', 'mean_legal_bert_sim', 'avg_conjunction_count',
       'avg_noun_count', 'frequency_index', 'n_long_words', 'avg_adverb_count',
       'automated_readability_index', 'lex_mtld', 'lex_maas',
       'avg_n_difficult_words', 'lex_yulek', 'flesch_reading_ease',
       'avg_syllables_per_word', 'lex_mattr', 'polysyllabcount',
       'syntactic_similarity', 'lex_msttr', 'mean_sentence_t5_base_sim',
       'mean_biobert_large_cased_v11_squad_sim', 'avg_adposition_count',
       'sentence_count', 'words_before_root', 'average_sentence_length',
       'char_count', 'lex_yulei', 'reading_time', 'mean_biobert_v11_sim',
       'n_difficult_words', 'lex_vocd', 'n_unique_words', 'lex_dugast',
       'imp_pos_ratio', 'smog_index', 'mean_biobert_base_cased_v12_sim',
       'gunnin

In [775]:
updated_new_columns3 = set(new_columns3) -{'USApolicyCitation'}
# print(len(new_columns3))
# len(updated_new_columns3)

####  Spearman's correlation

In [865]:
correlated_features = set()
correlation_matrix = scaled_merged_file3[updated_new_columns3].corr(method='spearman')

for i in range(len(correlation_matrix.columns)):
    for j in range(i):
        if abs(correlation_matrix.iloc[i, j]) >= 0.7:
            colname = correlation_matrix.columns[i]
            correlated_features.add(colname)
            # print(colname,"\n",correlated_features)
correlation_matrix_cols = set(correlation_matrix.columns)
selected_features_spearmanr = list(correlation_matrix_cols-correlated_features)

print("Selected Features:")
print(selected_features_spearmanr,"\n\nTotal : ", len(selected_features_spearmanr))
print(correlated_features)

Selected Features:
['lex_herdanvm', 'avg_verb_count', 'avg_word_length', 'word_count', 'flesch_kincaid_grade', 'lex_rttr', 'mcalpine_eflaw', 'pos_dissimilarity', 'mean_legal_bert_sim', 'avg_conjunction_count', 'frequency_index', 'avg_noun_count', 'avg_adverb_count', 'lex_mtld', 'syntactic_similarity', 'mean_sentence_t5_base_sim', 'avg_adposition_count', 'linsear_write_formula', 'avg_adjective_count', 'words_before_root'] 

Total :  20
{'lex_hdd', 'dale_chall_readability_score', 'lex_cttr', 'automated_readability_index', 'n_long_words', 'lex_maas', 'avg_n_difficult_words', 'lex_yulek', 'flesch_reading_ease', 'avg_syllables_per_word', 'lex_mattr', 'polysyllabcount', 'lex_msttr', 'mean_biobert_large_cased_v11_squad_sim', 'sentence_count', 'average_sentence_length', 'char_count', 'lex_yulei', 'reading_time', 'mean_biobert_v11_sim', 'n_difficult_words', 'lex_vocd', 'n_unique_words', 'lex_dugast', 'imp_pos_ratio', 'smog_index', 'mean_biobert_base_cased_v12_sim', 'gunning_fog', 'mean_biobert_

In [863]:
# Calculate Spearman correlation matrix among predictors
# keep an eye on USApolicyCitation and successfulPolicyPercentage

correlation_matrix = scaled_merged_file3[updated_new_columns3].corr(method='spearman')

# Set the diagonal elements (correlation with self) to zero
np.fill_diagonal(correlation_matrix.values, 0)

# Set the threshold for correlation (choose as per requirement)
threshold = 0.7

# Identify highly correlated features
highly_correlated_features = set()
for col in correlation_matrix.columns:
    
#     print(col)
#     print(correlation_matrix[col].abs())
          
    correlated_cols = correlation_matrix.index[correlation_matrix[col].abs() >= threshold].tolist()
    highly_correlated_features.update(correlated_cols)
    
    # print(correlated_cols)
    # print(highly_correlated_features)
# print("Highly Correlated Features:")
# print(highly_correlated_features)
correlation_matrix_cols = set(correlation_matrix.columns)
selected_features_spearmanr = list(correlation_matrix_cols-highly_correlated_features)

print("Selected Features:")
print(selected_features,"\n\nTotal : ", len(selected_features_spearmanr))

Selected Features:
['lex_herdanvm', 'avg_verb_count', 'avg_word_length', 'word_count', 'flesch_kincaid_grade', 'lex_rttr', 'mcalpine_eflaw', 'pos_dissimilarity', 'mean_legal_bert_sim', 'avg_conjunction_count', 'frequency_index', 'avg_noun_count', 'avg_adverb_count', 'lex_mtld', 'syntactic_similarity', 'mean_sentence_t5_base_sim', 'avg_adposition_count', 'linsear_write_formula', 'avg_adjective_count', 'words_before_root'] 

Total :  11


In [ ]:
# Spearman's Correlation
def display_correlation(df):
    r = df.corr(method="spearman")
    plt.figure(figsize=(10,6))
    heatmap = sns.heatmap(r, vmin=-1, 
                      vmax=1, annot=True)
    plt.title("Spearman Correlation")
    return(r)


def display_corr_pairs(df,color="cyan"):
    s = set_title = np.vectorize(lambda ax,r,rho: ax.title.set_text("r = " + 
                                        "{:.2f}".format(r) + 
                                        '\n $\\rho$ = ' + 
                                        "{:.2f}".format(rho)) if ax!=None else None
                            )      

    r = display_correlation(df)
    rho = df.corr(method="spearman")
    g = sns.PairGrid(df,corner=True)
    g.map_diag(plt.hist,color="yellow")
    g.map_lower(sns.scatterplot,color="magenta")
    set_title(g.axes,r,rho)
    plt.subplots_adjust(hspace = 0.6)
    plt.show()    
    
display_corr_pairs(scaled_merged_file3[selected_features_spearmanr])

In [ ]:
print("lexical_diversity_mltd: ",TRUNAJOD.ttr.d_estimate(doc, min_range = 35, max_range = 50, trials = 5) )

In [56]:
# text = (
#     "Many years later, as he faced the firing squad, Colonel Aureliano Buendía "
#      "was to remember that distant afternoon when his father took him to discover ice. "
#      "At that time Macondo was a village of twenty adobe houses, built on the bank "
#      "of a river of clear water that ran along a bed of polished stones, which were "
#      "white and enormous, like prehistoric eggs. The world was so recent "
#      "that many things lacked names, and in order to indicate them it was necessary to point."
#  )



doc = textacy.make_spacy_doc(output1, lang="en_core_web_sm")
print(doc._.preview)

ts1 = ts.TextStats(doc)
print("n_chars: ",ts1.n_chars)
print("n_syllables: ",ts1.n_syllables)
print("n_monosyllable_words: ",ts1.n_monosyllable_words)

print("n_words: ",ts1.n_words)
print("n_unique_words: ",ts1.n_unique_words)
print("n_long_words: ", ts1.n_long_words) #ts.basics.n_words(doc)
print("no of tokens: ",len(doc)) #no of tokens
print("n_sents: ",ts1.n_sents)#n_sents(doc)
print("\n")
#readability
print("flesch_kincaid_grade_level: ", ts1.flesch_kincaid_grade_level)
#10.922857142857143
print("automated_readability_index: ",ts1.automated_readability_index)
print("flesch_reading_ease: ",ts1.flesch_reading_ease)
print("smog_index: ",ts1.smog_index)
print("\n")
#lexical-diversity
# https://pypi.org/project/lexical-diversity/
flt = ld.flemmatize(output)
print("no of tokens: ",len(ld.tokenize(output)))
print("ttr: ",ld.ttr(flt))
print("root_ttr: ",ld.root_ttr(flt))
print("log_ttr: ",ld.log_ttr(flt))
print("maas_ttr: ",ld.maas_ttr(flt))
print("mtld: ", ld.mtld(flt))
print("hdd: ", ld.hdd(flt))
print("msttr: ", ld.msttr(flt,window_length=25))
print("mattr: ", ld.mattr(flt,window_length=25))
print("mtld_ma_wrap: ",ld.mtld_ma_wrap(flt))
print("mtld_ma_bid: ",ld.mtld_ma_bid(flt))


# print(ts1.entropy)
# print("ttr: ",ts1.ttr)
#0.7857142857142857
#ts2 = ts.readability(doc)

#doc._.to_bag_of_words()
#print(next(extract.words(doc)))
# print(extract.keyterms.textrank(doc))
# #preprocessing.remove.punctuation(text, only=[".", "?", "!"])

# next(iter(doc)) #tokens
# ts.readability
# ts.basics
#print(np.mean(ts1.n_chars_per_word))

# print(np.mean(ts1.n_syllables_per_word))

# print(ts1.counts)
# #doc.lang_


Doc(4273 tokens: "open or its open he epidemic related to an incr...")
n_chars:  21467
n_syllables:  6599
n_monosyllable_words:  2396
n_words:  3997
n_unique_words:  713
n_long_words:  1433
no of tokens:  4273
n_sents:  269


flesch_kincaid_grade_level:  9.68656830913148
automated_readability_index:  11.295732803319957
flesch_reading_ease:  52.07977769572537
smog_index:  12.488271264903705


no of tokens:  5571
ttr:  0.18739903069466882
root_ttr:  13.98730095641165
log_ttr:  0.805860748435703
maas_ttr:  0.05182667264787474
mtld:  60.832912291790876
hdd:  0.8501844379167769
msttr:  0.8641441441441438
mattr:  0.8650658013341067
mtld_ma_wrap:  65.04559325076288
mtld_ma_bid:  64.57289505385768


'\nDoc(4213 tokens: "open or its open he epidemic related to an incr...")\nn_chars:  21196\nn_syllables:  6525\nn_monosyllable_words:  2348\nn_words:  3938\nn_unique_words:  690\nn_long_words:  1428\nno of tokens:  4213\nn_sents:  267\n\n\nflesch_kincaid_grade_level:  9.713937777118371\nautomated_readability_index:  11.295765964205486\nflesch_reading_ease:  51.688214848884314\nsmog_index:  12.497200199523625\n\n\nno of tokens:  3937\nttr:  0.16535433070866143\nroot_ttr:  10.37524309552979\nlog_ttr:  0.7826012614032555\nmaas_ttr:  0.06046974575978705\nmtld:  41.75758664087188\nhdd:  0.7861842407255152\nmsttr:  0.8201273885350322\nmattr:  0.8163250702785934\nmtld_ma_wrap:  44.32816865633731\nmtld_ma_bid:  43.98230315465504'

In [57]:
#!pip3 install textstat
text=output1

In [58]:
import textstat  # https://pypi.org/project/textstat/

print("\n")
print('syllable_count:               ' + str(textstat.syllable_count(text)))
print('lexicon_count:                ' + str(textstat.lexicon_count(text, removepunct=True)))
print('sentence_count:               ' + str(textstat.sentence_count(text)))
print('char_count:                   ' + str(textstat.char_count(text, ignore_spaces=True)))
print('letter_count:                 ' + str(textstat.letter_count(text, ignore_spaces=True)))
print('polysyllabcount:              ' + str(textstat.polysyllabcount(text)))
print('monosyllabcount:              ' + str(textstat.monosyllabcount(text)))
print("\n")
print('*********************************************************************************')
#print("\n")
print('Flesch Reading Ease:          ' + str(textstat.flesch_reading_ease(text)))
print('Flesch Kincaid Grade:         ' + str(textstat.flesch_kincaid_grade(text)))
print('Smog Index:                   ' + str(textstat.smog_index(text)))
print('Coleman Liau Index:           ' + str(textstat.coleman_liau_index(text)))
print('Automated Readability Index:  ' + str(textstat.automated_readability_index(text)))
print('Dale Chall Readability Score: ' + str(textstat.dale_chall_readability_score(text)))
print('Difficult Words:              ' + str(textstat.difficult_words(text)))
print('Linsear Write Formula:        ' + str(textstat.linsear_write_formula(text)))
print('Gunning Fog:                  ' + str(textstat.gunning_fog(text)))
print('Text Standard:                ' + str(textstat.text_standard(text)))

print('mcalpine_eflaw:               ' + str(textstat.mcalpine_eflaw(text)))
print('reading_time:                 ' + str(textstat.reading_time(text, ms_per_char=14.69)))

print("\n")

print('*********************************************************************************')
"""Flesch-Kincaid Grade Level = This is a grade formula in that a score of 9.3 means that a ninth grader would be able to read the document"""
"""Gunning Fog = This is a grade formula in that a score of 9.3 means that a ninth grader would be able to read the document."""
"""SMOG - for 30 sentences or more  =This is a grade formula in that a score of 9.3 means that a ninth grader would be able to read the document."""
"""Automated Readability Index = Returns the ARI (Automated Readability Index) which outputs a number that approximates the grade level needed to comprehend the text."""
""" Coleman Liau Index = Returns the grade level of the text using the Coleman-Liau Formula."""
"""Linsear = Returns the grade level using the Linsear Write Formula."""
""" Dale Chall = Different from other tests, since it uses a lookup table of the most commonly used 3000 English words. Thus it returns the grade level using the New Dale-Chall Formula."""
""" mcalpine_eflaw = Returns a score for the readability of an english text for a foreign learner or English, focusing on the number of miniwords and length of sentences.
It is recommended to aim for a score equal to or lower than 25."""

'''



syllable_count:               6914
lexicon_count:                3996
sentence_count:               279
char_count:                   21743
letter_count:                 21464
polysyllabcount:              899
monosyllabcount:              2374


*********************************************************************************
Flesch Reading Ease:          48.5
Flesch Kincaid Grade:         10.0
Smog Index:                   13.4
Coleman Liau Index:           13.27
Automated Readability Index:  11.4
Dale Chall Readability Score: 6.21
Difficult Words:              405
Linsear Write Formula:        12.333333333333334
Gunning Fog:                  8.11
Text Standard:                12th and 13th grade
mcalpine_eflaw:               19.9
reading_time:                 319.4


*********************************************************************************


'\nsyllable_count:               6834\nlexicon_count:                3937\nsentence_count:               278\nchar_count:                   21471\nletter_count:                 21193\npolysyllabcount:              895\nmonosyllabcount:              2332\n\n\n*********************************************************************************\nFlesch Reading Ease:          48.6\nFlesch Kincaid Grade:         10.0\nSmog Index:                   13.4\nColeman Liau Index:           13.33\nAutomated Readability Index:  11.3\nDale Chall Readability Score: 6.15\nDifficult Words:              396\nLinsear Write Formula:        12.166666666666666\nGunning Fog:                  8.09\nText Standard:                12th and 13th grade\nmcalpine_eflaw:               19.7\nreading_time:                 315.41\n '

In [133]:
# https://pypi.org/project/py-readability-metrics/
# !pip3 install py-readability-metrics
#!python -m nltk.downloader punkt

from readability import Readability

r = Readability(text)

print("flesch_kincaid: ", r.flesch_kincaid())
# fk = r.flesch_kincaid()
# print(fk.score)
# print(fk.grade_level)

print("flesch reading ease: ", r.flesch())
# f = r.flesch()
# print(f.score)
# print(f.ease)
# print(f.grade_levels)

print("gunning_fog: ", r.gunning_fog())
# gf = r.gunning_fog()
# print(gf.score)
# print(gf.grade_level)

print("coleman_liau: ", r.coleman_liau())
# cl = r.coleman_liau()
# print(cl.score)
# print(cl.grade_level)

print("dale_chall: ", r.dale_chall())
# dc = dale_chall()
# print(dc.score)
# print(dc.grade_levels)

print("ari: ", r.ari())
# ari = r.ari()
# print(ari.score)
# print(ari.grade_levels)
# print(ari.ages)

'''
Linsear Write is a readability metric for English text, purportedly developed for the United States Air Force 
to help them calculate the readability of their technical manuals.
'''
print("linsear_write: ", r.linsear_write()) 
# lw = r.linsear_write()
# print(lw.score)
# print(lw.grade_level)

'''
The SMOG Readability Formula (Simple Measure of Gobbledygook) is a popular method to use on health literacy materials.
'''

print("smog: ", r.smog())
# s = r.smog()
# print(s.score)
# print(s.grade_level)

'''The original SMOG formula uses a sample of 30 sentences from the original text. 
However, the formula can be generalized to any number of sentences. 
You can use the generalized formula by passing the all_sentences=True argument to smog()'''
# s = r.smog(all_sentences=True)
# print(s.score)
# print(s.grade_level)

'''
flesch_kincaid:  score: 12.119517006828133, grade_level: '12'
flesch reading ease:  score: 32.91425886480454, ease: 'difficult', grade_levels: ['college']
gunning_fog:  score: 17.331488965919107, grade_level: 'college_graduate'
coleman_liau:  score: 13.80722377444755, grade_level: '14'
dale_chall:  score: 10.643150986866681, grade_levels: ['college_graduate']
ari:  score: 11.161216681109835, grade_levels: ['12'], ages: [17, 18]
linsear_write:  score: 11.413602941176471, grade_level: '11'
smog:  score: 15.731716950459138, grade_level: 16
'''
